In [2]:
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from pymatgen.ext.matproj import MPRester
from datetime import datetime
import traceback
from time import sleep
#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.by import By

In [4]:
with MPRester("aU621ICtBbnsLKk9") as mpr:
    start_time = datetime.now()
    print(mpr.get_data("mp-4793"))
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

[{'energy': -35.39037599, 'energy_per_atom': -7.0780751980000005, 'volume': 85.5157639613103, 'formation_energy_per_atom': -0.579842158000001, 'nsites': 5, 'unit_cell_formula': {'U': 1.0, 'Si': 2.0, 'Pd': 2.0}, 'pretty_formula': 'U(SiPd)2', 'is_hubbard': False, 'elements': ['U', 'Si', 'Pd'], 'nelements': 3, 'e_above_hull': 0, 'hubbards': {}, 'is_compatible': True, 'spacegroup': {'symprec': 0.1, 'source': 'spglib', 'symbol': 'I4/mmm', 'number': 139, 'point_group': '4/mmm', 'crystal_system': 'tetragonal', 'hall': '-I 4 2'}, 'task_ids': ['mp-917621', 'mp-927697', 'mp-927260', 'mp-4793', 'mp-1144071', 'mp-1266548', 'mp-1439283', 'mp-1595594', 'mp-1671157'], 'band_gap': 0.0, 'density': 9.845665171877243, 'icsd_id': None, 'icsd_ids': [648881, 108867, 57468, 603822, 648883, 77900], 'cif': "# generated using pymatgen\ndata_U(SiPd)2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   5.92642027\n_cell_length_b   5.92642027\n_cell_length_c   5.92642027\n_cell_angle_alpha   139.92176015\n_c

In [ ]:
import pandas as pd
df = pd.read_csv("raw_data_from_PhD_paper.csv")
print(df)
materialsGathered = df['mp_id'].tolist()
namesOfMaterial = df['formula'].tolist()
print(materialsGathered)
print(len(materialsGathered))

In [4]:
#https://materialsproject.org/materials?_limit=75&_skip=150
site= "https://next-gen.materialsproject.org/materials?_limit=75&_skip=75"
end = "https://next-gen.materialsproject.org/materials?_limit=75&_skip=146250"
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import html5lib
from random import randint
import time
width = 1024
height = 768

chrome_options = Options()
chrome_options.page_load_strategy = 'normal'
chrome_options.add_argument('--enable-automation')
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--lang=en')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-insecure-localhost')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-browser-side-navigation')
chrome_options.add_argument('--mute-audio')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--force-device-scale-factor=1')
chrome_options.add_argument(f'window-size={width}x{height}')

chrome_options.add_experimental_option(
    'prefs', {
        'intl.accept_languages': 'en,en_US',
        'download.prompt_for_download': False,
        'download.default_directory': '/dev/null',
        'automatic_downloads': 2,
        'download_restrictions': 3,
        'notifications': 2,
        'media_stream': 2,
        'media_stream_mic': 2,
        'media_stream_camera': 2,
        'durable_storage': 2,
    }
)

driver = webdriver.Chrome(r'C:\Users\dylan\Desktop\chromedriver.exe', options=chrome_options) # Optional argument, if not specified will search path.
driver.get(site)
driver.set_page_load_timeout(60)
username = "ionQubit"
password = "Aa08008321"
driver.get("https://profile.materialsproject.org/connect/github")
time.sleep(1)
#driver.get("https://www.phptravels.net/login")
el = driver.find_element(By.XPATH, "//*[@id='login_field']")
el.send_keys(username)
el = driver.find_element(By.XPATH, "//*[@id='password']")
el.send_keys(password)
time.sleep(1)
element = driver.find_element(By.NAME, "commit")
time.sleep(1)
element.click()
materials = []
print("Running...")
for i in range(0, 146250, 75):
    print(i)
    if(i == 0):
        time.sleep(4)
    success = False
    while(success is False):
        driver.get("https://next-gen.materialsproject.org/materials?_limit=75&_skip=" + str(i))
        time.sleep(3)
        content = driver.page_source
        soup = BeautifulSoup(content, 'html.parser')
        for link in soup.find_all('a'):
            if(link.get('href') != None and "mp" in link.get('href')):
                materials.append(link.get('href')[14:].split("?")[0])
                success = True
driver.quit()

C:\Users\dylan\AppData\Local\Temp\ipykernel_13076\2354879120.py:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\dylan\Desktop\chromedriver.exe', options=chrome_options) # Optional argument, if not specified will search path.


Running...
0
75
150
225
300
375
450
525
600
675
750
825
900
975
1050
1125
1200
1275
1350
1425
1500
1575
1650
1725
1800
1875
1950
2025
2100
2175
2250
2325
2400
2475
2550
2625
2700
2775
2850
2925
3000
3075
3150
3225
3300
3375
3450
3525
3600
3675
3750
3825
3900
3975
4050
4125
4200
4275
4350
4425
4500
4575
4650
4725
4800
4875
4950
5025
5100
5175
5250
5325
5400
5475
5550
5625
5700
5775
5850
5925
6000
6075
6150
6225
6300
6375
6450
6525
6600
6675
6750
6825
6900
6975
7050
7125
7200
7275
7350
7425
7500
7575
7650
7725
7800
7875
7950
8025
8100
8175
8250
8325
8400
8475
8550
8625
8700
8775
8850
8925
9000
9075
9150
9225
9300
9375
9450
9525
9600
9675
9750
9825
9900
9975
10050
10125
10200
10275
10350
10425
10500
10575
10650
10725
10800
10875
10950
11025
11100
11175
11250
11325
11400
11475
11550
11625
11700
11775
11850
11925
12000
12075
12150
12225
12300
12375
12450
12525
12600
12675
12750
12825
12900
12975
13050
13125
13200
13275
13350
13425
13500
13575
13650
13725
13800
13875
13950
14025
14100
14175


KeyboardInterrupt: 

In [1]:
index = 0
entries = []
value = []
structs = []
with MPRester("aU621ICtBbnsLKk9") as m:
    for i in materials:
        try:
            validity = m.get_phonon_bandstructure_by_material_id("mp-" + i).has_imaginary_freq()
            structure = m.get_structure_by_material_id("mp-" + i)
            structs.append(structure)
            if validity:
                values.append(0)
            else:
                values.append(1)
            entries.append("mp-" + str(i))
            if(i % 10 == 0):
                print("Progress: ", i/len(materials) * 100, "%")
                print(len(structs))
        except:
            pass

NameError: name 'MPRester' is not defined

In [ ]:
df['final_structure'] = structs

In [ ]:
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
df['wyckoff'] = df['final_structure'].apply(get_aflow_label_spglib)
df['composition'] = df['final_structure'].apply(lambda x: x.composition.reduced_formula)

In [ ]:
print(df)

In [ ]:
print(df.keys())
df = df.drop(['final_structure'], axis=1)
df["bulk_inversion"] = df["bulk_inversion"].astype(int)

In [ ]:
df.to_csv('C:\\Users\\dylan\\Desktop\\generatedData\\predictProperties.csv', index=False)